In [ ]:
# @title 1. 下載套件
!pip install -q rich
!pip install -q pyngrok
!pip install -q line-bot-sdk
!pip install -q langchain-groq
!pip install langchain-core

# 安裝所需的 Groq 函式庫
!pip install -q groq

# 注意：如果您的 Colab 環境有 GPU，可以將 faiss-cpu 改為 faiss-gpu 以獲得更好效能
!pip install -q langchain langchain-community langchain-huggingface faiss-cpu sentence-transformers
print("必要的函式庫包含已安裝完畢。")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# @title 2. 導入需要的工具
import os
from google.colab import userdata
from rich.pretty import pprint
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#新增images資料夾

In [ ]:
# #刪除images的圖片
# import shutil
# import os

# image_folder = 'images'

# if os.path.exists(image_folder):
#     try:
#         shutil.rmtree(image_folder)
#         print(f"資料夾 '{image_folder}' 及其內容已成功刪除。")
#     except OSError as e:
#         print(f"刪除資料夾 '{image_folder}' 時發生錯誤：{e}")
# else:
#     print(f"資料夾 '{image_folder}' 不存在，無需刪除。")

資料夾 'images' 及其內容已成功刪除。


In [ ]:
# #將圖片下載到雲端硬碟
# !cp -r images /content/drive/MyDrive/

In [3]:
# @title 4. 匯入、設定金鑰、常數與模型初始化
import os # 保留 os 模組，以防未來其他地方用到
# from groq import Groq
# from langchain_groq import ChatGroq
# 確保匯入測試區塊可能需要的模組
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
# 匯入 Colab Secrets 功能
# from google.colab import userdata

# # --- API 金鑰設定 (使用 Colab Secrets) ---
# GROQ_API_KEY = None # 先初始化為 None
# try:
#     # 從 Colab Secrets 讀取名為 'GROQ_API_KEY' 的密鑰
#     GROQ_API_KEY = userdata.get('GROQ_API_KEY')

#     if not GROQ_API_KEY:
#         # 如果密鑰不存在、值為空或 Notebook access 未啟用，userdata.get 會返回 None
#          print("⚠️ 警告：未能從 Colab Secrets 獲取 GROQ_API_KEY。")
#          print("   請檢查以下設定：")
#          print("   1. 在 Colab 左側的鑰匙(🔑)圖示面板中，是否已建立名稱完全為 'GROQ_API_KEY' 的 Secret？")
#          print("   2. 該 Secret 的『值』(Value) 欄位是否已填入您的 Groq API Key？")
#          print("   3. 該 Secret 的 'Notebook access' (允許這個筆記本存取) 開關是否已啟用 (ON)？")
#          # 此處不拋出錯誤，讓後續的檢查來處理 GROQ_API_KEY 為 None 的情況

# except Exception as e:
#     # 處理讀取 Secret 過程中的其他潛在錯誤 (雖然較少見)
#     print(f"讀取 Colab Secret 'GROQ_API_KEY' 時發生非預期錯誤：{e}")
#     GROQ_API_KEY = None # 確保出錯時變數為 None

# --- 常數設定 ---
# LLM_MODEL_NAME = "llama3-70b-8192"  # 或是您要使用的其他模型

# --- 初始化 Groq Client ---
# client = None
# if GROQ_API_KEY: # 只有在成功獲取 API Key 後才嘗試初始化
#     try:
#         # 使用獲取到的 API Key 初始化 Groq client
#         client = Groq(api_key=GROQ_API_KEY)
#         print("✅ Groq client 初始化成功。")
#     except Exception as e:
#         print(f"❌ 初始化 Groq client 時發生錯誤：{e}")
#         print("   請檢查您的 API Key 格式是否正確，或是否有網路連線問題。")
#         client = None # 確保初始化失敗時 client 為 None
# else:
#     print("❌ 由於未能獲取 GROQ_API_KEY，無法嘗試初始化 Groq client。")

# --- 初始化 Langchain 的 ChatGroq 模型 ---
llm = OllamaLLM(
    model="llama3.3:latest",   #  model名稱
    base_url="http://125.228.146.37:11434"
)
# 只有在 Groq client 成功初始化後才進行 (這也間接確認了 API Key 存在且初步有效)
# if client:
#     try:
#         llm = ChatGroq(model=LLM_MODEL_NAME, groq_api_key=GROQ_API_KEY)
#         print("✅ Langchain ChatGroq 模型初始化成功。")
#     except Exception as e_llm:
#         print(f"❌ 初始化 Langchain ChatGroq 模型時發生錯誤：{e_llm}")
#         llm = None # 確保初始化失敗時 llm 為 None
# elif not GROQ_API_KEY:
#      print("❌ 由於未獲取 API Key，Langchain ChatGroq 模型無法初始化。")
# else: # API Key 存在，但 client 初始化失敗
#      print("❌ 由於 Groq client 初始化失敗，Langchain ChatGroq 模型無法初始化。")


# --- 除錯訊息 ---
print("\n--- 初始化狀態 ---")
# print(f"GROQ_API_KEY 設定: {'是' if GROQ_API_KEY else '否'}") # 現在反映是否成功從 Secrets 取得
# print(f"Groq client 初始化: {'成功' if client else '失敗'}")
print(f"LLM 模型初始化: {'成功' if llm else '失敗'}")
# print(f"使用的模型名稱: {LLM_MODEL_NAME}")
print("------------------\n")

# --- 範例測試 (可選) ---
if llm:
    print("執行 API 連線與 Langchain 整合測試...")
    try:
        # # 測試 Groq API
        # response = client.chat.completions.create(
        #     model=LLM_MODEL_NAME,
        #     messages=[{"role": "user", "content": "你好！簡單回答即可。"}]
        # )
        # print("   - Groq API 連線測試成功。")
        # print(response.choices[0].message.content) # 可取消註解查看回應

        # 測試 Langchain 整合
        prompt_template = ChatPromptTemplate.from_template("用一句話總結 Langchain。")
        chain = prompt_template | llm | StrOutputParser()
        langchain_response = chain.invoke({})
        print("   - Langchain 整合測試成功。")
        # print(langchain_response) # 可取消註解查看回應

    except Exception as test_error:
        print(f"\n--- 測試失敗 ---")
        print(f"   測試過程中發生錯誤：{test_error}")
        print("   請檢查您的 API 金鑰有效性、網路連線，以及模型名稱是否正確。")
        print("------------------\n")
else:
    print("\n--- 由於 client 或 llm 未成功初始化，跳過測試。 ---")
    print("---------------\n")


--- 初始化狀態 ---
LLM 模型初始化: 成功
------------------

執行 API 連線與 Langchain 整合測試...
   - Langchain 整合測試成功。


In [ ]:
# --- 必要的函式庫 ---
print("正在載入必要的函式庫...")
from flask import Flask, request, abort, send_from_directory
from pyngrok import ngrok, conf # type: ignore
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
import time
import traceback
import os
import re
from google.colab import userdata # type: ignore
# Langchain / Groq imports
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# 嘗試匯入 jieba
try:
    import jieba # type: ignore
    print("   - jieba 函式庫載入成功。")
except ImportError:
    jieba = None
    print("   - 警告: 未安裝 jieba 函式庫，將使用基本分詞。建議 pip install jieba")

print("函式庫載入完成。")

# --- 常數設定 ---
SIMPLIFIED_MD_FILENAME = "simplified_output_by_section.md"
TARGET_DESCRIPTION_KEYWORDS = ["結塊", "過篩", "順序", "吸濕", "稠度", "黏稠", "流動性"] # 仍可輔助識別，但搜尋主要靠原料
CHINESE_STOP_WORDS = {"的", "和", "與", "或", "了", "呢", "嗎", "喔", "啊", "關於", "有關", "請", "請問", " ", ""}
ALLOWED_WORKSHEET_IDENTIFIERS = [
    "工作表: 9", "工作表: 10"
]

# --- SOP 查詢相關函式 ---

def load_markdown_sections(filename=SIMPLIFIED_MD_FILENAME):
    """讀取並解析 Markdown 檔案"""
    sections = []; print(f"正在嘗試載入檔案: {filename}")
    try:
        with open(filename, 'r', encoding='utf-8') as f: markdown_content = f.read()
        print(f"檔案 {filename} 讀取成功。")
    except FileNotFoundError: print(f"❌ 錯誤：找不到檔案 '{filename}'。"); return []
    except Exception as e: print(f"❌ 讀取檔案 '{filename}' 時發生錯誤：{e}"); return []
    parts = re.split(r'(## 工作表:.*)', markdown_content)
    for i in range(1, len(parts), 2):
        if i + 1 < len(parts):
            title = parts[i].strip(); content = parts[i + 1].strip().removeprefix("Here is the simplified content:").strip().removeprefix("Here is the simplified output:").strip()
            if title and content: sections.append({"title": title, "content": content})
        elif i < len(parts):
            title = parts[i].strip()
            if title: sections.append({"title": title, "content": ""})
    if not sections: print(f"⚠️ 警告：未能從檔案 '{filename}' 解析出任何工作表區塊。")
    return sections

def filter_sections_by_title(all_sections, allowed_identifiers=ALLOWED_WORKSHEET_IDENTIFIERS):
    """根據標題關鍵字過濾區塊列表"""
    if not all_sections: return []
    return [sec for sec in all_sections if any(allowed_id in sec.get("title", "") for allowed_id in allowed_identifiers)]

# --- *** 修改：基於規則的關鍵字提取函式 (更側重原料) *** ---
def extract_keywords_rule_based(user_input, target_keywords=TARGET_DESCRIPTION_KEYWORDS):
    """使用規則拆分和比對關鍵字，主要提取原料名稱。"""
    print(f"--- (階段0) 使用規則解析輸入 (主要提取原料): '{user_input}' ---")
    input_cleaned = user_input.strip().lower()
    if not input_cleaned:
        return None

    if jieba:
        tokens = list(jieba.cut_for_search(input_cleaned))
    else:
        input_no_punct = re.sub(r'[^\w\s]', ' ', input_cleaned)
        tokens = input_no_punct.split()
    print(f"   分詞結果: {tokens}")

    potential_materials = []
    identified_characteristics = set() # 仍然記錄特性詞，但它們不作為主要搜尋依據

    for token in tokens:
        token_clean = token.strip()
        if not token_clean or token_clean in CHINESE_STOP_WORDS:
            continue

        is_characteristic = False
        # 檢查是否為特性關鍵字 (完全匹配)
        for target_char in target_keywords:
            if token_clean == target_char.lower():
                identified_characteristics.add(target_char) # 記錄原始大小寫的特性關鍵字
                is_characteristic = True
                break

        # 如果不是特性詞，且不是數字，且長度大於0，則視為潛在原料
        if not is_characteristic:
            if not token_clean.isnumeric() and len(token_clean) > 0:
                potential_materials.append(token_clean)

    materials_list_unique = sorted(list(set(potential_materials)))
    characteristics_list_sorted = sorted(list(identified_characteristics))

    if not materials_list_unique:
        print("⚠️ 規則解析器：未能識別出任何潛在的原料名稱。")
        # 即使沒有原料，也返回結構，讓後續判斷
        # return {"原料名稱": [], "特性描述": characteristics_list_sorted}
        return None # 或者直接返回None，如果嚴格要求必須有原料名才能查詢

    result = {"原料名稱": materials_list_unique, "特性描述": characteristics_list_sorted}
    print(f"   規則解析結果 (主要為原料，輔助特性): {result}")
    return result
# --- *** 函式結束 *** ---

def search_sections(sections_to_search, keywords):
    """在已過濾的區塊中初步篩選包含【原料名稱】關鍵字的工作表"""
    relevant_sections = []
    # *** 修改：只使用原料名稱進行初步搜尋 ***
    material_keywords = keywords.get("原料名稱", [])
    all_keywords = [kw for kw in material_keywords if kw and isinstance(kw, str)]

    if not all_keywords:
        print("ℹ️ 沒有有效的【原料名稱】關鍵字可供搜尋。")
        return []
    print(f"DEBUG: 正在使用以下任一【原料名稱】關鍵字搜尋: {all_keywords}")
    for section in sections_to_search:
        text_to_search = section.get("title", "") + section.get("content", "")
        if any(keyword.lower() in text_to_search.lower() for keyword in all_keywords): # 忽略大小寫搜尋
            relevant_sections.append(section)
    return relevant_sections

# --- *** 修改：第一階段 LLM Prompt，強調小範圍、僅原料相關 *** ---
def extract_relevant_text(llm, sections, keywords):
    """(第一階段 LLM) 使用極度聚焦的 Prompt 提取與指定【原料名稱】最直接相關的【小範圍文字片段】。"""
    material_names = keywords.get('原料名稱', [])
    if not material_names:
        print("ℹ️ extract_relevant_text: 查詢中未提供原料名稱。")
        return []
    material_name_str = "、".join(material_names)

    # 特性描述可以作為輔助上下文，幫助LLM理解，但提取焦點是原料
    characteristics_list = keywords.get('特性描述', [])
    description_keywords_str = ', '.join(characteristics_list)

    if characteristics_list:
        # 雖然主要搜原料，但告知LLM使用者也提到了這些特性，可能有助於LLM理解上下文
        keywords_str_display_for_prompt = f"主要查詢的原料名稱：【{material_name_str}】。(使用者同時提及了這些相關詞彙，供參考：{description_keywords_str})"
    else:
        keywords_str_display_for_prompt = f"主要查詢的原料名稱：【{material_name_str}】"

    prompt_base = """
        你的任務是作為一個精確的文字提取器。
        在下方提供的「工作表內容」中，僅找出與「主要查詢的原料名稱」最直接相關的【一個或多個簡短文字片段、句子或列表項】。

        {keywords_str_display_for_prompt}

        工作表內容：
        ```markdown
        {text}
        ```

        輸出要求：
        1.  精確定位到包含「主要查詢的原料名稱」的句子、操作步驟或其非常緊密的上下文。
        2.  **【直接輸出】** 找到的相關內容。提取的範圍應盡可能小而精準。例如，僅提取包含該原料名稱的那個步驟、那句描述或相關的列表項。
        3.  **【避免提取】** 過長的段落或整個子標題下的所有內容，除非該原料名稱在整個段落/子標題中都高度相關且內容不可分割。優先提取最小的有效資訊單位。
        4.  **【絕對禁止】** 包含 '## 工作表: ...' 這個最高級別的標題。
        5.  **【絕對禁止】** 包含任何 '製表日期', '製表人' 等頁腳資訊。
        6.  **【絕對禁止】** 進行任何摘要、改寫或添加任何【解釋性文字、前言、結語】(例如不要說 "以下是找到的內容：" 或 "根據關鍵字..." 等)。輸出結果必須直接就是提取的內容本身。
        7.  如果在此「工作表內容」中找不到直接與「主要查詢的原料名稱」相關的內容，【只輸出】以下固定文字：「在此文件中未找到與此原料直接相關的內容」。
        8.  必須使用【繁體中文】輸出 (如果提取的內容本身是中文)。
        再次強調：直接輸出提取的、與原料直接相關的【小範圍】內容，不要加任何其他文字。
        """

    # 使用帶有所有預期輸入變量的模板
    prompt_template = ChatPromptTemplate.from_template(prompt_base)
    chain = prompt_template | llm | StrOutputParser()

    relevant_texts = []
    # 顯示一次完整的 keywords_str_display_for_prompt，因為它對於所有 section 都是一樣的
    print(f"--- (階段1) 使用 LLM 定位並提取子章節，使用的查詢上下文: {keywords_str_display_for_prompt} ---")

    for section in sections:
        section_title = section.get("title", "無標題區塊")
        section_content = section.get("content", "")
        section_full_text = f"{section_content}" # 簡化，只傳遞內容讓 LLM 專注

        print(f"   正在處理區塊: {section_title} (搜尋原料 '{material_name_str}')...")
        try:
            relevant_text = chain.invoke({
                "keywords_str_display_for_prompt": keywords_str_display_for_prompt,
                "text": section_full_text
            })
            relevant_text = relevant_text.strip()

            # 基本的後處理，移除 LLM 可能不小心加上的 markdown 標籤或引導詞
            if relevant_text.startswith("根據關鍵字") or relevant_text.startswith("以下是找到"):
                lines = relevant_text.splitlines()
                relevant_text = "\n".join(lines[1:]).strip()
            if relevant_text.startswith("```markdown"):
                relevant_text = relevant_text.removeprefix("```markdown").removesuffix("```").strip()

            is_found_message = "在此文件中未找到與此原料直接相關的內容" in relevant_text

            if is_found_message or not relevant_text: # 如果是 "未找到" 或提取結果為空
                print(f"     ↳ 在區塊 '{section_title}' 中未找到或提取到空內容關於原料 '{material_name_str}'。LLM回應: '{relevant_text}'")
                relevant_texts.append({"title": section_title, "text": relevant_text if relevant_text else "內容提取為空", "found": False})
            else:
                print(f"     ↳ 從 '{section_title}' 提取到內容 (前100字): \"{relevant_text[:100].replace(os.linesep, ' ')}...\"")
                relevant_texts.append({"title": section_title, "text": relevant_text, "found": True})

        except Exception as e:
            print(f"❌ 從區塊 '{section_title}' 提取時出錯: {e}")
            traceback.print_exc()
            relevant_texts.append({"title": section_title, "text": "LLM 提取失敗", "found": False})
    return relevant_texts
# --- *** 函式結束 *** ---

# --- *** 修改：第二階段 LLM Prompt，強調原文呈現、不修飾、統一列表 *** ---
def synthesize_results(llm, keywords, extracted_texts):
    """(第二階段 LLM) 將提取出的【小範圍文字片段】整合成【極簡且原文呈現的統一格式列表】。"""
    if not extracted_texts:
        return "未能提取到任何相關內容以供整合。"

    valid_extractions = [
        item['text'] for item in extracted_texts
        if item.get("found", False) and \
           "LLM 提取失敗" not in item['text'] and \
           "在此文件中未找到與此原料直接相關的內容" not in item['text'] and \
           item['text'] and item['text'].strip() # 確保文本本身不為空或僅包含空白
    ]

    if not valid_extractions:
        material_names_list = keywords.get('原料名稱', [])
        material_name_str = "、".join(material_names_list) if material_names_list else "所查詢的項目"
        # 檢查是否所有嘗試提取的區塊都返回了 "未找到" 或類似訊息
        all_attempted_but_not_found = all(
            not item.get("found") or \
            "在此文件中未找到與此原料直接相關的內容" in item['text'] or \
            item['text'].strip() == ""
            for item in extracted_texts
        )
        if extracted_texts and all_attempted_but_not_found: # 確保 extracted_texts 不是空的
            return f"已檢查所有相關SOP文件區塊，但均未找到關於原料【{material_name_str}】的直接操作說明或注意事項。"
        return f"雖然初步定位到可能相關的SOP區塊，但在其中未能找到關於原料【{material_name_str}】的具體操作說明或注意事項。"


    print(f"\n🔄 (階段2) 正在整合 {len(valid_extractions)} 份提取的重點內容並統一格式 (力求原文、簡潔)...")
    # 將所有提取片段組合，用特殊分隔符明確告知LLM這是不同來源的獨立片段
    combined_extracted_text = "\n\n片段分隔線 (請將每個片段視為獨立資訊來源)\n\n".join(valid_extractions)

    material_names_list = keywords.get('原料名稱', [])
    material_name = "、".join(material_names_list) if material_names_list else "未指定原料"

    characteristics_list = keywords.get('特性描述', [])
    if characteristics_list:
        keywords_str_for_prompt = f"使用者主要查詢的原料名稱為【{material_name}】。(使用者查詢時提及的相關詞彙，供您理解上下文：{', '.join(characteristics_list)})"
    else:
        keywords_str_for_prompt = f"使用者主要查詢的原料名稱為【{material_name}】。"

    synthesis_prompt_template_str = """
        您是一位SOP內容整理員。您的任務是將下方提供的、已從SOP文件中提取出的、與指定原料相關的【多個獨立的簡短文字片段】，整理成一個【極簡的、統一格式的數字編號列表】。

        {keywords_str_for_prompt}

        已提取的相關SOP片段 (這些片段來自不同地方，請將它們視為獨立的資訊點，使用「片段分隔線」隔開)：
        ---
        {combined_extracted_text}
        ---

        您的任務與輸出要求：
        1.  仔細閱讀所有「已提取的相關SOP片段」。每個片段都是關於上述「主要查詢的原料名稱」的直接相關內容。
        2.  **【核心任務】：將這些片段中的【每一個獨立的資訊點、操作步驟、或注意事項】整理出來，作為列表中的一個獨立項目。**
        3.  **【格式統一】：** 使用從 1 開始的數字編號列表 (格式如：1., 2., 3., ...)。
        4.  **【原文呈現】：** 盡最大可能【直接使用】提取片段中的【原文表述】。**【嚴格禁止】任何形式的改寫、摘要、解釋、歸納或添加您自己的文字或評論。** 您的工作是「彙整原文」和「格式化為列表」，而不是「再創作」或「解釋」。
        5.  如果原始片段本身就是一個短列表或包含子步驟，請在新的數字編號下，盡可能保持其原始結構，例如使用縮排和 '-' 或 '*' 來表示層級關係。
        6.  **【極簡輸出】：** 您的最終輸出【必須直接是這個數字編號列表本身】。**【嚴格禁止】** 包含任何前言（例如 "好的，這是整理後的列表：" 或 "關於您查詢的..."）、標題、引導語或結語。
        7.  如果多個片段描述的是【完全相同或幾乎完全重複】的資訊，請只選擇其中一個最清晰的表述放入列表，以避免不必要的冗餘。但若有些微差異或補充，寧可保留兩者，避免資訊遺失。**判斷重複的標準要非常嚴格。**
        8.  使用**繁體中文**。
        9.  即使最終整理出來的資訊點很少（例如只有一兩條），也直接按列表格式輸出。
        10. **【絕對禁止】** 在輸出中提及任何「工作表標題」、「SOP文件來源」或「片段分隔線」等元信息。
        11. **【絕對禁止】** 添加任何超出所提供片段內容之外的資訊或建議。

        請直接開始輸出列表：
        """
    synthesis_prompt_template = ChatPromptTemplate.from_template(synthesis_prompt_template_str)
    synthesis_chain = synthesis_prompt_template | llm | StrOutputParser()

    try:
        final_response = synthesis_chain.invoke({
            "keywords_str_for_prompt": keywords_str_for_prompt,
            "combined_extracted_text": combined_extracted_text
        })
        final_response = final_response.strip()

        # 後處理：強力移除已知的前綴 (LLM有時還是會忍不住加上)
        unwanted_prefixes = [
            "好的，這是整理後的列表：", "這是為您整理的列表：", "以下是整理後的列表：",
            "根據您提供的資訊：", "關於您查詢的原料", "關於您查詢的",
            "Okay, here is the list:", "Here is the list:", "Here is the output:",
            "這是您查詢的結果：", "這是相關的資訊：", "查詢結果如下：",
            "1. " # 有時LLM會自己加列表編號，但我們的prompt要求它從1開始，所以如果它以 "1. " 開頭，通常是正確的
        ]
        # 移除前綴的邏輯需要小心，避免誤刪正常的列表開頭
        cleaned_response = final_response
        for prefix in unwanted_prefixes:
            # 確保 prefix 不是 "1. " 這種可能與正確輸出衝突的
            if prefix.strip().startswith("1.") and cleaned_response.strip().startswith("1."):
                continue
            if final_response.lower().startswith(prefix.lower()):
                # 計算前綴實際長度，移除此前綴及其後的空白和冒號
                prefix_len = len(prefix)
                temp_response = final_response[prefix_len:].lstrip("：: ").strip()
                # 只有當移除後還有內容，或者移除的是明確的引導語時才更新
                if temp_response or prefix.lower() not in ["1."]: # 避免 "1. " 被錯誤移除後變空
                    cleaned_response = temp_response
                    print(f"   DEBUG: (Synthesize) 移除了不需要的前綴 '{prefix}'")
                    break # 找到並移除一個就不再檢查其他前綴

        # 再一次檢查是否以數字列表開頭，如果不是，且內容看起來像引言，嘗試移除
        lines = cleaned_response.splitlines()
        if lines and not re.match(r"^\s*\d+\.\s*", lines[0]): # 如果第一行不是 "數字." 開頭
            # 並且第一行看起來像是一句簡短的引言而不是實際內容
            first_line_lower = lines[0].lower()
            is_likely_intro = True
            # 如果第一行包含原料名，可能不是引言，除非非常短
            for mat_kw in keywords.get("原料名稱", []):
                if mat_kw.lower() in first_line_lower:
                    if len(lines[0]) > 20 : # 如果包含原料名但較長，可能就是內容
                        is_likely_intro = False
                    break
            if len(lines[0]) > 50: # 如果第一行很長，不像引言
                is_likely_intro = False

            if is_likely_intro and len(lines) > 1 : # 確保有多行，移除後還有內容
                print(f"   DEBUG: (Synthesize) 輸出首行 '{lines[0]}' 並非標準列表項，嘗試移除作為引言。")
                cleaned_response = "\n".join(lines[1:]).strip()
            elif is_likely_intro and len(lines) == 1: # 只有一行且不像列表項
                print(f"   DEBUG: (Synthesize) 輸出只有一行 '{lines[0]}' 且並非標準列表項，可能LLM未遵循指示。")
                # 此時可以考慮返回一個固定訊息或原始提取內容的簡單組合
                # cleaned_response = "LLM未能按要求格式化資訊，請重試或檢查日誌。"

        if not cleaned_response.strip() and valid_extractions:
            print(f"   ⚠️ 警告: (Synthesize) LLM 最終輸出為空或僅含空白，但之前有 {len(valid_extractions)} 條有效提取內容。可能整合失敗。")
            return f"已找到關於原料【{material_name}】的相關資訊，但在最終整理格式時出現問題。請稍後再試。"

        # 如果清理後 response 變空，但 valid_extractions 有內容，說明 LLM 可能完全沒按指示輸出
        if not cleaned_response and valid_extractions:
             return f"已找到關於原料【{material_name}】的相關資訊片段，但無法按預期格式化呈現。請稍後重試。"


        return cleaned_response

    except Exception as e:
        print(f"❌ 整合結果時發生嚴重錯誤：{e}")
        traceback.print_exc()
        return "整合結果時發生嚴重錯誤，請檢查系統日誌。"
# --- *** 函式結束 *** ---

# --- 全域變數 ---
llm = None
sections_to_search = []
line_bot_api = None
handler = None
public_url = None
LLM_MODEL_NAME = "llama3-70b-8192" # 或者您使用的模型
app = Flask(__name__)
initialization_success = False # 新增一個全域變數來追蹤初始化狀態

# --- 訊息處理函式定義 ---
def handle_message(event):
    global llm, sections_to_search, line_bot_api, initialization_success
    msg = event.message.text
    reply_token = event.reply_token
    user_id = event.source.user_id
    print(f"收到來自 {user_id} 的訊息: '{msg}'")
    start_time = time.time()
    reply_text = "抱歉，處理您的請求時發生內部問題，請稍後再試。" # 預設錯誤訊息

    if not initialization_success or not llm or not sections_to_search: # 檢查初始化狀態
        reply_text = "系統正在初始化或遇到啟動問題，請稍後再試。如果問題持續，請聯繫管理員。"
        print("❌ 錯誤：系統未完全初始化 (LLM 或 SOP sections 未就緒，或初始化過程失敗)。")
    else:
        try:
            # --- 使用修改後的規則解析輸入 ---
            keywords_extraction_result = extract_keywords_rule_based(msg, TARGET_DESCRIPTION_KEYWORDS) # Stage 0

            if keywords_extraction_result and keywords_extraction_result.get("原料名稱"):
                # 主要使用原料名稱進行搜尋
                # search_query_keywords 傳給 search_sections
                # keywords_extraction_result (包含原料和可能的特性) 傳給 LLM 階段作為完整上下文

                # ** 修改：直接使用 keywords_extraction_result 給 search_sections，讓其內部決定用哪些關鍵字 **
                # search_sections 內部已修改為只用 "原料名稱"
                relevant_sections = search_sections(sections_to_search, keywords_extraction_result) # Stage 1 Search

                if relevant_sections:
                    print(f"   初步找到 {len(relevant_sections)} 個可能相關區塊，針對查詢: {keywords_extraction_result.get('原料名稱')}")
                    # 傳遞完整的 keywords_extraction_result 給 LLM，讓 LLM 有更完整的上下文
                    extracted_texts = extract_relevant_text(llm, relevant_sections, keywords_extraction_result) # Stage 2 Extract
                    final_summary = synthesize_results(llm, keywords_extraction_result, extracted_texts) # Stage 3 Synthesize
                    reply_text = final_summary
                else:
                    material_name_str = "、".join(keywords_extraction_result.get("原料名稱", ["未知原料"]))
                    reply_text = f"在指定的工作表範圍內，找不到與原料【{material_name_str}】直接相關的SOP內容。"
                    print(f"   在指定範圍內未找到與原料【{material_name_str}】相關的區塊。")
            else:
                reply_text = "無法從您的訊息中解析出有效的原料名稱進行查詢。請嘗試輸入明確的原料名。"
                print("   未能從輸入解析出有效的原料名稱 (規則解析失敗或無原料結果)。")
        except Exception as e:
            print(f"!!!!!!!!!! 處理訊息 '{msg}' 時發生嚴重錯誤 !!!!!!!!!!")
            traceback.print_exc()
            reply_text = "抱歉，處理您的查詢時遇到未預期的錯誤，工程師已收到通知，請稍後再試。"

    end_time = time.time()
    processing_time = end_time - start_time
    print(f"訊息 \"{msg}\" 處理完成，耗時 {processing_time:.2f} 秒。最終回覆 (前100字): {reply_text[:100].replace(os.linesep, ' ')}")
    try:
        if line_bot_api:
            # 檢查 reply_text 是否為空，如果是空，給一個通用訊息
            if not reply_text or not reply_text.strip():
                print("⚠️警告：最終回覆內容為空，將發送通用提示訊息。")
                reply_text = "抱歉，未能找到明確的資訊或處理時發生問題。請嘗試更換查詢詞或稍後再試。"

            messages_to_reply = [TextSendMessage(text=reply_text)]
            line_bot_api.reply_message(reply_token, messages=messages_to_reply)
            print(f"   ✅ 成功回覆訊息至 LINE。")
        else:
            print("❌ 錯誤：line_bot_api 未初始化，無法回覆訊息。")
    except Exception as e:
        print(f"❌ 回覆訊息至 LINE 時發生錯誤: {e}")
        traceback.print_exc()

# --- 初始化設定 ---
def initialize_bot():
    global llm, sections_to_search, line_bot_api, handler, public_url, initialization_success, app, LLM_MODEL_NAME # 加入 initialization_success
    print("--- 開始執行初始化設定 ---")
    initialization_success = False # 重置狀態
    LINE_ACCESS_TOKEN = LINE_SECRET = GROQ_API_KEY = NGROK_AUTH_TOKEN = None
    # 1. 載入 API 金鑰
    print("1. 載入 API 金鑰...")
    try:
        LINE_ACCESS_TOKEN = userdata.get('LINE_ACCESS_TOKEN')
        LINE_SECRET = userdata.get('LINE_SECRET')
        GROQ_API_KEY = userdata.get('GROQ_API_KEY')
        NGROK_AUTH_TOKEN = userdata.get('ngrok_token')
        if not all([LINE_ACCESS_TOKEN, LINE_SECRET, GROQ_API_KEY]): # NGROK_AUTH_TOKEN is optional for local dev
            print("❌ 錯誤：缺少必要的 LINE 或 GROQ API 金鑰。")
            return False # 直接返回，不再繼續
        else:
            print("   ✅ API 金鑰載入成功。")
    except Exception as e:
        print(f"❌ 從 Colab Secrets 載入金鑰時發生錯誤: {e}")
        return False

    # 2. 初始化 LLM
    print(f"2. 初始化 LLM ({LLM_MODEL_NAME})...")
    try:
        llm = ChatGroq(model=LLM_MODEL_NAME, groq_api_key=GROQ_API_KEY, temperature=0.05) # 溫度可以調低，讓輸出更穩定
        print(f"   ✅ LLM ({LLM_MODEL_NAME}) 初始化成功。")
    except Exception as e:
        print(f"❌ 初始化 LLM ({LLM_MODEL_NAME}) 時發生錯誤: {e}")
        llm = None # 確保 llm 是 None
        return False

    # 3. 載入並過濾 SOP 文件區塊
    print("3. 載入並過濾 SOP 文件區塊...")
    all_sections = load_markdown_sections()
    if all_sections:
        original_count = len(all_sections)
        sections_to_search = filter_sections_by_title(all_sections)
        if sections_to_search:
            allowed_titles_str = ', '.join([s.get('title','?') for s in sections_to_search])
            print(f"   ✅ 成功載入 {original_count} 區塊，已過濾出 {len(sections_to_search)} 個目標區塊: [{allowed_titles_str}]")
        else:
            print(f"❌ 警告：雖然載入SOP文件，但未能找到任何指定的工作表 {ALLOWED_WORKSHEET_IDENTIFIERS}。系統仍可啟動，但查詢範圍可能受限。")
            # 即使找不到特定工作表，也允許繼續，但 sections_to_search 會是空的
            # 或者，可以選擇在這裡 return False 如果嚴格要求必須有這些工作表
    else:
        print("❌ 錯誤：未能載入任何 SOP 文件區塊。系統無法處理查詢。")
        sections_to_search = [] # 確保是空列表
        return False

    # 4. 初始化 LINE Bot API
    print("4. 初始化 LINE Bot...")
    try:
        line_bot_api = LineBotApi(LINE_ACCESS_TOKEN)
        handler = WebhookHandler(LINE_SECRET)
        # 注意：handle_message 不再使用 @handler.add 裝飾器，而是在這裡手動註冊
        # handler.add(MessageEvent, message=TextMessage)(handle_message) # 這是舊方法
        # 新方法：定義一個 wrapper 或直接傳遞
        @handler.add(MessageEvent, message=TextMessage)
        def message_handler_wrapper(event):
            handle_message(event) # 呼叫我們的主處理函數
        print("   ✅ LINE Bot Handler 初始化完成，並已註冊訊息處理器。")
    except Exception as e:
        print(f"❌ 初始化 LineBotApi 或 WebhookHandler 時發生錯誤: {e}")
        line_bot_api = None
        handler = None
        return False

    # 5. 設定並啟動 Ngrok (如果需要)
    # 檢查是否在 Colab 環境或有 NGROK_AUTH_TOKEN 才啟動 ngrok
    run_ngrok = 'google.colab' in str(get_ipython()) if 'get_ipython' in globals() else False # type: ignore
    if not run_ngrok and NGROK_AUTH_TOKEN: # 如果不在Colab但有token，也可能想用ngrok
        run_ngrok = True
    elif not NGROK_AUTH_TOKEN and run_ngrok: # 在colab但沒有token
        print("   ⚠️ 警告: 在Colab環境但未設定Ngrok認證權杖(ngrok_token)，ngrok通道可能不穩定或受限。")


    if run_ngrok:
        print("5. 設定並啟動 Ngrok..."); flask_port = int(os.environ.get("PORT", 5000))
        try:
            if NGROK_AUTH_TOKEN:
                conf.get_default().auth_token = NGROK_AUTH_TOKEN # 新版 pyngrok 設定方式
                print("   - Ngrok 認證權杖已設定。")
            # ngrok.kill() # 可能需要 try-except 包裹，如果沒有正在運行的通道會報錯
            try: ngrok.kill()
            except: pass

            # 檢查是否有PYTHONASYNCIODEBUG環境變量，如果有，ngrok.connect可能會出問題
            if os.environ.get('PYTHONASYNCIODEBUG'):
                print("   ⚠️ 偵測到 PYTHONASYNCIODEBUG 環境變量，暫時移除以避免影響 ngrok。")
                original_async_debug = os.environ.pop('PYTHONASYNCIODEBUG')
                public_url = ngrok.connect(flask_port).public_url
                os.environ['PYTHONASYNCIODEBUG'] = original_async_debug # 恢復
            else:
                public_url = ngrok.connect(flask_port).public_url

            print(f"   ✅ ngrok 通道已啟動: \"{public_url}\" -> \"[http://127.0.0.1](http://127.0.0.1):{flask_port}\"")
            print(f"   👉 請將 LINE Webhook URL 設為: {public_url}/") # LINE Webhook 通常不需要路徑，直接根目錄
        except Exception as e:
            print(f"❌ 啟動 ngrok 時發生錯誤: {e}")
            traceback.print_exc()
            if "ERR_NGROK_4018" in str(e) or "authentication failed" in str(e).lower():
                print("   >>> Ngrok 認證失敗，請檢查 Colab Secret 'ngrok_token' 或 Ngrok 儀表板設定。<<<")
            public_url = None # 確保 public_url 是 None
            # Ngrok 失敗不一定代表整個服務失敗，如果只是本地測試可以不用 ngrok
            # 但對於 LINE Bot，ngrok 通常是必要的
            return False # 如果 ngrok 是必要的，這裡應該返回 False
    else:
        print("5. 跳過 Ngrok 啟動 (可能為本地開發或不需要外部訪問)。")
        public_url = None


    initialization_success = True # 所有步驟成功
    print("--- 初始化設定成功完成 ---")
    return True

# --- Webhook 端點 ---
@app.route("/", methods=['POST'])
def callback():
    global handler
    if not handler:
        print("❌ Callback 錯誤：Webhook Handler 未初始化。")
        abort(500)
    signature = request.headers.get('X-Line-Signature', '')
    body = request.get_data(as_text=True)
    # print(f"Webhook received. Signature: {signature[:10]}... Body: {body[:100]}...") # Debug
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("❌ 錯誤：LINE Webhook 簽名驗證失敗。請檢查您的 Channel Secret 是否正確。")
        abort(400)
    except AttributeError as ae:
        print(f"❌ 錯誤：Handler 物件不存在或無效 (AttributeError): {ae}")
        traceback.print_exc()
        abort(500)
    except Exception as e:
        print(f"❌ 處理 Webhook 時發生未預期錯誤: {e}")
        traceback.print_exc()
        abort(500)
    return 'OK'

# --- 執行 Flask App ---
if __name__ == "__main__":
    # 檢查是否在 Colab Notebook 環境中
    is_colab = 'google.colab' in str(get_ipython()) if 'get_ipython' in globals() else False # type: ignore

    if initialize_bot():
        print("\n--- 所有元件已就緒，正在準備啟動 Flask app ---")
        port = int(os.environ.get("PORT", 5000))
        if is_colab and public_url:
            print(f"Flask app 將在背景運行。LINE Webhook URL: {public_url}/")
            # 在 Colab 中，app.run() 會阻塞，但 ngrok 已經啟動。
            # 通常 Colab Cell 執行完畢後，背景服務可能終止。
            # 這裡讓它運行，但使用者需要知道如何停止 (通常是中斷 kernel)。
            # app.run(host='0.0.0.0', port=port) # 這樣可以在 Colab 中運行
            print(f"在 Colab 環境中，Flask app (port {port}) 已配置。請確保此 Cell 持續運行以保持服務。")
            # 若要在 Colab 中真正後台運行，需要更複雜的處理或使用 `nohup` 等（但不適用於 Notebook cell）
            # 簡單起見，告知用戶保持 Cell 運行
            try:
                 app.run(host='0.0.0.0', port=port) # 如果在 Colab，ngrok 已經處理了外部訪問
            except KeyboardInterrupt:
                 print("\n--- Flask app 已手動停止 (KeyboardInterrupt) ---")
            except Exception as run_err:
                 print(f"\n--- Flask app 運行時發生錯誤: {run_err} ---")
                 traceback.print_exc()
            finally:
                if public_url:
                    try: ngrok.disconnect(public_url); print("   - ngrok 通道已關閉。")
                    except Exception as ng_err: print(f"   - 關閉 ngrok 通道 '{public_url}' 時發生錯誤: {ng_err}")
                try: ngrok.kill(); print("   - 所有 ngrok 通道已嘗試關閉。")
                except Exception as ng_kill_err: print(f"   - 嘗試關閉所有 ngrok 通道時發生錯誤: {ng_kill_err}")
                print("--- 程式結束 ---")

        else: # 非 Colab 環境或 ngrok 未啟動
            print(f"Flask app 準備在本地端口 {port} 啟動...")
            if public_url:
                 print(f"LINE Webhook URL 應指向: {public_url}/")
            else:
                 print("Ngrok 未啟動或配置失敗。如果需要 LINE Bot 連接，請確保本地服務可被外部訪問。")
            try:
                app.run(host='0.0.0.0', port=port, debug=False) # debug=True 可能導致 ngrok 問題和重複初始化
            except KeyboardInterrupt:
                print("\n--- Flask app 已手動停止 ---")
            except Exception as run_err:
                print(f"\n--- Flask app 運行時發生錯誤: {run_err} ---")
                traceback.print_exc()
            finally:
                if public_url: # 確保即使在非Colab環境也關閉ngrok
                    try: ngrok.disconnect(public_url); print("   - ngrok 通道已關閉。")
                    except Exception as ng_err: print(f"   - 關閉 ngrok 通道 '{public_url}' 時發生錯誤: {ng_err}")
                try: ngrok.kill(); print("   - 所有 ngrok 通道已嘗試關閉。")
                except Exception as ng_kill_err: print(f"   - 嘗試關閉所有 ngrok 通道時發生錯誤: {ng_kill_err}")
                print("--- 程式結束 ---")
    else:
        print("\n❌ 因初始化失敗，無法啟動 Flask app。請檢查上方錯誤訊息。")

# 在 Colab Cell 中，您需要執行此 Cell 來啟動服務。
# 若要停止服務，請點擊 Cell 左側的停止按鈕或中斷 Kernel。

正在載入必要的函式庫...
   - jieba 函式庫載入成功。
函式庫載入完成。
--- 開始執行初始化設定 ---
1. 載入 API 金鑰...
   ✅ API 金鑰載入成功。
2. 初始化 LLM (llama3-70b-8192)...
   ✅ LLM (llama3-70b-8192) 初始化成功。
3. 載入並過濾 SOP 文件區塊...
正在嘗試載入檔案: simplified_output_by_section.md
檔案 simplified_output_by_section.md 讀取成功。
   ✅ 成功載入 23 區塊，已過濾出 2 個目標區塊: [## 工作表: 9.新品-樣品製作注意事項SOP, ## 工作表: 10.新品-原料使用注意事項]
4. 初始化 LINE Bot...
   ✅ LINE Bot Handler 初始化完成，並已註冊訊息處理器。
5. 設定並啟動 Ngrok...
   - Ngrok 認證權杖已設定。


<ipython-input-4-6e18aa3f33ff>:482: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(LINE_ACCESS_TOKEN)
<ipython-input-4-6e18aa3f33ff>:483: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(LINE_SECRET)


   ✅ ngrok 通道已啟動: "https://8db1-35-225-61-135.ngrok-free.app" -> "[http://127.0.0.1](http://127.0.0.1):5000"
   👉 請將 LINE Webhook URL 設為: https://8db1-35-225-61-135.ngrok-free.app/
--- 初始化設定成功完成 ---

--- 所有元件已就緒，正在準備啟動 Flask app ---
Flask app 將在背景運行。LINE Webhook URL: https://8db1-35-225-61-135.ngrok-free.app/
在 Colab 環境中，Flask app (port 5000) 已配置。請確保此 Cell 持續運行以保持服務。
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/May/2025 23:00:33] "POST / HTTP/1.1" 200 -
Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...


收到來自 Ub0b9891b0b6e502ebb21a0a2c19a95eb 的訊息: '奶酪'
--- (階段0) 使用規則解析輸入 (主要提取原料): '奶酪' ---


...

   ✅ 成功回覆訊息至 LINE。
收到來自 Ub0b9891b0b6e502ebb21a0a2c19a95eb 的訊息: '奶精'
--- (階段0) 使用規則解析輸入 (主要提取原料): '奶精' ---
   分詞結果: ['奶精']
   規則解析結果 (主要為原料，輔助特性): {'原料名稱': ['奶精'], '特性描述': []}
DEBUG: 正在使用以下任一【原料名稱】關鍵字搜尋: ['奶精']
   初步找到 1 個可能相關區塊，針對查詢: ['奶精']
--- (階段1) 使用 LLM 定位並提取子章節，使用的查詢上下文: 主要查詢的原料名稱：【奶精】 ---
   正在處理區塊: ## 工作表: 9.新品-樣品製作注意事項SOP (搜尋原料 '奶精')...
     ↳ 從 '## 工作表: 9.新品-樣品製作注意事項SOP' 提取到內容 (前100字): "c.  原料本身吸濕性高：     ★ 奶精：放第二段，非常容易吸濕不可以放在第一段，因流動性好不須過篩。..."

🔄 (階段2) 正在整合 1 份提取的重點內容並統一格式 (力求原文、簡潔)...
訊息 "奶精" 處理完成，耗時 1.16 秒。最終回覆 (前100字): 1. 奶精：放第二段，非常容易吸濕不可以放在第一段，因流動性好不須過篩。


<ipython-input-4-6e18aa3f33ff>:421: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(reply_token, messages=messages_to_reply)
INFO:werkzeug:127.0.0.1 - - [09/May/2025 23:04:22] "POST / HTTP/1.1" 200 -


   ✅ 成功回覆訊息至 LINE。
收到來自 Ub0b9891b0b6e502ebb21a0a2c19a95eb 的訊息: '色素'
--- (階段0) 使用規則解析輸入 (主要提取原料): '色素' ---
   分詞結果: ['色素']
   規則解析結果 (主要為原料，輔助特性): {'原料名稱': ['色素'], '特性描述': []}
DEBUG: 正在使用以下任一【原料名稱】關鍵字搜尋: ['色素']
   初步找到 1 個可能相關區塊，針對查詢: ['色素']
--- (階段1) 使用 LLM 定位並提取子章節，使用的查詢上下文: 主要查詢的原料名稱：【色素】 ---
   正在處理區塊: ## 工作表: 9.新品-樣品製作注意事項SOP (搜尋原料 '色素')...
     ↳ 從 '## 工作表: 9.新品-樣品製作注意事項SOP' 提取到內容 (前100字): "色素粉的製作方式有二種，a.直接加入液體香料b.使用PG溶解 a.  秤取液體香料後再加入色素，仔細將色粉粒搓開後再浸泡。 b.  因為香料本身溶劑的不同，有些香料無法溶解色素，此時需使用PG溶色素。..."

🔄 (階段2) 正在整合 1 份提取的重點內容並統一格式 (力求原文、簡潔)...
訊息 "色素" 處理完成，耗時 10.73 秒。最終回覆 (前100字): 1. 色素粉的製作方式有二種，a.直接加入液體香料b.使用PG溶解 2. a. 秤取液體香料後再加入色素，仔細將色粉粒搓開後再浸泡。 3. b. 因為香料本身溶劑的不同，有些香料無法溶解色素，此時需使


<ipython-input-4-6e18aa3f33ff>:421: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(reply_token, messages=messages_to_reply)
INFO:werkzeug:127.0.0.1 - - [09/May/2025 23:04:50] "POST / HTTP/1.1" 200 -


   ✅ 成功回覆訊息至 LINE。
